This notebook contains an example of using `redbiom` through it's Python API to extract a subset of American Gut Project samples. These data are then loaded into QIIME 2 for a mini beta-diversity analysis using UniFrac. This assumes we're using a QIIME 2 2018.11 environment that additionally has `redbiom` 0.3.0 installed. The exact commands I ran to install it are:

```
$ conda install nltk
$ pip install https://github.com/biocore/redbiom/archive/0.3.0.zip
```

In [1]:
import redbiom.summarize
import redbiom.search
import redbiom.fetch
import qiime2

The first thing we're going to do is gather the `redbiom` contexts. A context is roughly a set of consistent technical parameters. For example, the specific sequenced gene, the variable region within the gene, the length of the read, and how the operational taxonomic units were assessed.

The reason `redbiom` partitions data into contexts is because these technical details can lead to massive technical bias. The intention is to facilitate comparing "apples" to "apples". 

The context summarization returns a pandas `DataFrame` so it should be pretty friendly to manipulate.

In [2]:
contexts = redbiom.summarize.contexts()

In [3]:
contexts.head()

,ContextName,SamplesWithData,FeaturesWithData,Description
0,Pick_closed-reference_OTUs-Greengenes-FLX-16S-...,20,21,Qiita context
1,Pick_closed-reference_OTUs-Greengenes-Illumina...,117725,70394,Qiita context
2,Deblur-Illumina-16S-V45-100nt-fbc5b2,213,76187,Deblur (Reference phylogeny for SEPP: Greengen...
3,Pick_closed-reference_OTUs-Greengenes-Titanium...,357,3736,Qiita context
4,Pick_closed-reference_OTUs-Greengenes-FLX-16S-...,58,1568,Qiita context


At the present time, determining the context to use is a bit manual and requires some strung munging. Additional development is needed.

Let's take a look at the larger contexts.

In [4]:
contexts.sort_values('SamplesWithData', ascending=False).head()

,ContextName,SamplesWithData,FeaturesWithData,Description
24,Pick_closed-reference_OTUs-Greengenes-Illumina...,156761,83309,Qiita context
15,Pick_closed-reference_OTUs-Greengenes-Illumina...,145669,74959,Qiita context
90,Deblur-Illumina-16S-V4-100nt-fbc5b2,141809,5698310,Deblur (Reference phylogeny for SEPP: Greengen...
17,Pick_closed-reference_OTUs-Greengenes-Illumina...,141380,72999,Qiita context
48,Deblur-Illumina-16S-V4-90nt-99d1d8,137678,4576516,Deblur (Reference phylogeny for SEPP: Greengen...


For simplicity sake, let's take the first context. It's large, and the phylogeny associated with the operational taxonomic units is easy to get. We'll break down the meaning of the context name in a moment. In practice, you will _most likely_ want to use the Deblur data, however producing a reasonable tree from those data requies a slightly computationally expensive step, and I'm on my laptop right now with limited battery quite literally in the middle of nowhere on a bus in the Czech Republic.

In [5]:
ctx = contexts.sort_values('SamplesWithData', ascending=False).iloc[0]['ContextName']

In [6]:
ctx

'Pick_closed-reference_OTUs-Greengenes-Illumina-16S-V4-5c6506'

Breaking this name into its constiuent pieces, this is a closed reference context meaning that operational taxonomic units were assessed against a reference database and sequences which did not recruit to the reference were discarded. The reference used is Greengenes, a common 16S reference database. The gene represented by the data is the 16S SSU rRNA gene, and specifically the V4 region of the gene. Finally, the fragments represented are truncated to 100 nucleotides. (Don't worry if this is all a lot of jargon. It is a lot of jargon. Please ask questions :)

So cool, we have a "context". What can we do now? Let's search for some sample identifiers based off of the metadata (i.e., variables) associated with the samples. Specifically, let's get some skin, oral and fecal samples. Be forewarned, the metadata search uses Python's `ast` module behind the scenes, so malformed queries at present produce tracebacks.

In [7]:
study_id = 10317  # the Qiita study ID of the American Gut Project is 10317
results = {}
for site in ['sebum', 'saliva', 'feces']:
    query = "where qiita_study_id==%d and env_material=='%s'" % (study_id, site)
    results[site] = redbiom.search.metadata_full(query)

In [8]:
for k, v in results.items():
    print(k, len(v))

feces 18659
saliva 1406
sebum 1289


As you can see, the sample sets are not very balanced. For the purposes of the mini-analysis, let's just grab the first 100 from each sample type.

In [9]:
to_keep = []
for k, v in results.items():
    to_keep.extend(list(v)[:100])

In [10]:
to_keep[:5]

['10317.000010759',
 '10317.000098944',
 '10317.000009227',
 '10317.000040400',
 '10317.000050016']

The last output cell shows what these IDs look like. These are Qiita sample IDs.

Now that we have some samples, let's get some data! What we're going to do is ask `redbiom` to obtain the sample data, for our `to_keep` samples, from the context we previously selected. What's happening behind the scenes is that the API is pulling out sparse vectors corresponding to the number of individual sequences observed for each operational taxonomic unit per sample, and additionally unmunging the names (as `redbiom` normalizes sample and feature identifiers). The output is then aggregated into what's called a BIOM `Table`, which is really just a rich object wrapped around a `scipy.sparse` matrix. 

You may noice two outputs on the return. The one we're ignoring represents "ambiguous" samples. Some sample identifiers are associated with multiple sequenced samples. This is because some samples may "fail" sequencing, where they didn't yield sufficient sequence data, and were rerun. These "failures" are still represented in Qiita, but are differentiated by the actual sequencing run they were on. This doesn't matter for the moment though.

In [11]:
biom_table, _ = redbiom.fetch.data_from_samples(ctx, to_keep)

In [12]:
biom_table

15321 x 305 <class 'biom.table.Table'> with 122392 nonzero entries (2% dense)

The `repr` output shows that we have roughly 13k OTUs (operational taxonomic units), and only 244 samples. What gives? We were supposed to get 300! Just because a sample has metadata does not mean it has sequence data. It is also possible that some of the samples haven't been run through the same bioinformatic processing (e.g., closed reference at 100nt).

More information on `biom` can be found [here](http://biom-format.org/). 

Let's play with the object for just a moment for familiarity.

In [13]:
biom_table.head()

5 x 5 <class 'biom.table.Table'> with 1 nonzero entries (4% dense)

In [14]:
print(biom_table.head())

# Constructed from biom file
#OTU ID	10317.000013687.2926	10317.000063464.33800	10317.000050729.46739	10317.000050928.46739	10317.000052477.26481
3711749	0.0	0.0	0.0	0.0	0.0
4363202	0.0	0.0	0.0	0.0	0.0
4446973	0.0	0.0	0.0	0.0	4.0
823745	0.0	0.0	0.0	0.0	0.0
691033	0.0	0.0	0.0	0.0	0.0


When we print the table, it implicitly casts it to a dense matrix representation. One thing to note: the sample IDs look different than before, right? The way that the individual sequencing runs are denoted is the last number -- the sample ID is represented by "qiita_study_id.the_actual_id.the_sequencing_run_id". This is to ensure the identifier is "globally" unique.

The identifiers on the rows are the "Greengenes" IDs. Also called OTU IDs. Or Feature IDs.

So we have a table. Now let's get the sample metadata. This will come down as a pandas `DataFrame`.

In [15]:
md, _ = redbiom.fetch.sample_metadata(to_keep, context=ctx)

In [16]:
md.head()

,#SampleID,anonymized_name,body_habitat,body_product,body_site,description,dna_extracted,env_biome,env_feature,env_material,env_package,host_common_name,host_subject_id,host_taxid,qiita_study_id,sample_type,scientific_name,taxon_id
25370_10317.000030028,10317.000030028.25370,000030028,UBERON:feces,UBERON:feces,UBERON:feces,American Gut Project Stool Sample,true,dense settlement biome,human-associated habitat,feces,human-gut,human,d43a43a39a32200b0141edde91b6ddc0aca0a8a11a7c3a...,9606,10317,Stool,human gut metagenome,408170
25370_10317.000054127,10317.000054127.25370,000054127,UBERON:feces,UBERON:feces,UBERON:feces,American Gut Project Stool Sample,true,dense settlement biome,human-associated habitat,feces,human-gut,human,8b872a357655894fe9f5708f91e513772ac88cdb7d4300...,9606,10317,Stool,human gut metagenome,408170
25501_10317.000031007,10317.000031007.25501,000031007,UBERON:feces,UBERON:feces,UBERON:feces,American Gut Project Stool Sample,true,dense settlement biome,human-associated habitat,feces,human-gut,human,6b1bfa9f343a48cbe6dc97ddd16cc7eaae1a43fe71f631...,9606,10317,Stool,human gut metagenome,408170
25501_10317.000036948,10317.000036948.25501,000036948,UBERON:feces,UBERON:feces,UBERON:feces,American Gut Project Stool Sample,true,dense settlement biome,human-associated habitat,feces,human-gut,human,2205f639ad8ca153aba08cb17391752160c615f56fe92f...,9606,10317,Stool,human gut metagenome,408170
25501_10317.000042695,10317.000042695.25501,000042695,UBERON:feces,UBERON:feces,UBERON:feces,American Gut Project Stool Sample,true,dense settlement biome,human-associated habitat,feces,human-gut,human,ae96c6187c3fb3766257cbf931012d063c5c999e3d36d6...,9606,10317,Stool,human gut metagenome,408170


In [17]:
md.sample_type.value_counts()

Stool         99
Mouth         98
Forehead      46
Right Hand    29
Left Hand     14
Skin           8
Torso          6
Left leg       3
Right leg      1
Hair           1
Name: sample_type, dtype: int64

Great! So we have a bunch of sample data, and some study variables! Now for the QIIMEing. What we'll need to do is pacakge these data into types that QIIME 2 understands. And in particular, we need to use the semantic type system. This is well documented on the QIIME 2 website, and I recommend reviewing there.

In [18]:
table_ar = qiime2.Artifact.import_data('FeatureTable[Frequency]', biom_table)
md_ar = qiime2.Metadata(md.set_index('#SampleID'))

Ah! We need a tree! Since we're using Greengenes, we can just rely on the existing prebuilt tree from the reference. Let's get that.

In [83]:
!wget ftp://ftp.microbio.me/greengenes_release/gg_13_8_otus/

--2019-04-08 20:48:41--  ftp://ftp.microbio.me/greengenes_release/gg_13_8_otus/
           => '.listing'
Resolving ftp.microbio.me... 169.228.46.98
Connecting to ftp.microbio.me|169.228.46.98|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /greengenes_release/gg_13_8_otus ... done.
==> PASV ... done.    ==> LIST ... done.

.listing                [ <=>                ]     511  --.-KB/s    in 0s      

2019-04-08 20:48:41 (1.75 MB/s) - '.listing' saved [511]

Removed '.listing'.
Wrote HTML-ized index to 'index.html' [1018].


In [86]:
!wget ftp://ftp.microbio.me/greengenes_release/gg_13_8_otus/trees/99_otus.tree

--2019-04-08 21:17:40--  ftp://ftp.microbio.me/greengenes_release/gg_13_8_otus/trees/99_otus.tree
           => '99_otus.tree'
Resolving ftp.microbio.me... 169.228.46.98
Connecting to ftp.microbio.me|169.228.46.98|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /greengenes_release/gg_13_8_otus/trees ... done.
==> SIZE 99_otus.tree ... 5171002
==> PASV ... done.    ==> RETR 99_otus.tree ... done.
Length: 5171002 (4.9M) (unauthoritative)

99_otus.tree        100%[===================>]   4.93M  4.15MB/s    in 1.2s    

2019-04-08 21:17:43 (4.15 MB/s) - '99_otus.tree' saved [5171002]



In [87]:
tree_ar = qiime2.Artifact.import_data('Phylogeny[Rooted]', '99_otus.tree')

Excellent. So next up, we need to rarefy these data. Rarefaction is a just randomly subsampling the samples without replacement in order to normalize for sequencing effort. There are a lot of ways to do this type of normalization. Rarefaction is dirty as you through out data, but it's pragmatic and tends to work pretty well. The issue rarefaction helps to ameliorate is that, the more you sequence the more life you'll observe. So if you don't do something to normalize sequencing effort across samples, you'll have a bad time.

Let's import all the plugins we'll be using.

In [88]:
from qiime2.plugins import feature_table, diversity, emperor

In [89]:
# rarefy to 1000 sequences per sample (yes, it's arbitrary)
rare_ar, = feature_table.actions.rarefy(table=table_ar, sampling_depth=1000)

save as biomtable

In [90]:
table_ar.export_data('full_otus')

In [91]:
!ls full_otus

feature-table.biom


In [92]:
import biom

In [93]:
btable = biom.load_table('full_otus/feature-table.biom')

## Skbio Grenegenes tree load

fix tree issue - https://forum.qiime2.org/t/greengenes-tree-download-with-branch-lengths/3329/5

In [94]:
import skbio

t = skbio.TreeNode.read('99_otus.tree')

for n in t.traverse():
    if n.length is None:
        n.length = 0.0


# import qiime2
# ar = qiime2.Artifact.import_data('Phylogeny[Rooted]', t)
# ar.save('97_otus.qza')

visualize tree - note this tree is huge so cannot output in notebook, so instead save to .txt file. Since the tree is so big its not worth visualizing

In [95]:
s = t.ascii_art()

text_file = open("greengene_tree.txt", "w")
text_file.write(s)
text_file.close()

calculate alpha diversity with qiime library

In [96]:
alpha_div = diversity.actions.alpha_phylogenetic(table=rare_ar, phylogeny=ar, metric='faith_pd')

In [97]:
alpha_div.alpha_diversity.export_data('alpha_div')

directly calculate alpha_div with skbio instead (I dont like biomTable structure, its confusing)

In [98]:
counts = btable.matrix_data.toarray().astype(int).T

In [99]:
sample_ids = btable.ids(axis='sample')
feature_ids = btable.ids(axis='observation')

In [100]:
result = skbio.diversity.alpha_diversity(metric='faith_pd',
                                         counts=counts,
                                         ids=sample_ids,
                                         otu_ids=feature_ids,
                                         tree=t)


In [102]:
print(len(result))
result[:5]

305


10317.000013687.2926     16.14610
10317.000063464.33800    33.90194
10317.000050729.46739    25.02934
10317.000050928.46739    52.32478
10317.000052477.26481    26.05148
dtype: float64

calculate beta diversity

In [103]:
unweighted_unifrac_ar, = diversity.actions.beta_phylogenetic(table=rare_ar, 
                                                             phylogeny=tree_ar, 
                                                             metric='unweighted_unifrac')

So before we move on, we should discuss what this voodoo just did. Beta diversity is what ecologists call comparing how similar (or dissimilar) two samples are. For example, how similar is a forest in the Pacific Northwest to a forest in Maine? (i.e., beta diversity is a function which takes two vectors and returns a scaler). We perform this calculation over all pairs of samples though so that we can examine all of the sample relationships. This distance matrix gets large quick: in our case, it's already 244 by 244. 

Visualizing that matrix would suck. So, one thing we often do is principal *coordinates* analysis. It's very similar to principal *components* analysis, except that we can pass in our distance matrix.

In [104]:
unweighted_unifrac_pcoa_ar, = diversity.actions.pcoa(unweighted_unifrac_ar)

//anaconda/envs/qiime2-2018.11/lib/python3.5/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:152: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -0.034632117913480066 and the largest is 16.846116422891065.
  RuntimeWarning


Finally, let's actually view the coordinates. It's much more interesting to use the metadata as well :) Hint: color by body site.

In [105]:
viz, = emperor.actions.plot(unweighted_unifrac_pcoa_ar, md_ar)

In [106]:
viz

<visualization: Visualization uuid: f9378991-94d7-4752-9b14-ae13b8f4aa6a>